In [212]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import math
from tqdm import tqdm

In [95]:
dataframe = pd.read_csv('data_full.csv')

/var/folders/cw/hgq_4j310_x0341w_fpbn62m0000gp/T/ipykernel_35408/854334815.py:1: DtypeWarning: Columns (23,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv('data_full.csv')


In [137]:
dataframe.head()

,ROW_NUM,USER_ID,INV_OPEN_DATE,INV_NUM_ACCOUNTS,INV_ACC_BAL,MONEY_ACCOUNT_OPEN_DATE,MONEY_ACCOUNT_STATUS,FICO,MONEY_DATA_UPDATE_DATE,MONEY_TOTAL_BALANCE,...,uses_relay,uses_loan,uses_credit,num_products,date_of_creating_new_columns,days_with_invest,days_with_money,days_with_relay,days_with_loan,days_with_credit
0,1.0,22398681.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,True,False,False,1,2023-03-15,NaN,NaN,318.0,NaN,NaN
1,1.0,22398688.0,2022-02-12,1.0,11686.97,NaT,NaN,NaN,NaN,NaN,...,False,False,False,1,2023-03-15,396.0,NaN,NaN,NaN,NaN
2,1.0,22398692.0,2022-02-12,1.0,29964.85,NaT,NaN,NaN,NaN,NaN,...,False,False,False,1,2023-03-15,396.0,NaN,NaN,NaN,NaN
3,1.0,22398695.0,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,True,False,False,1,2023-03-15,NaN,NaN,105.0,NaN,NaN
4,1.0,22398697.0,2022-02-12,2.0,507874.07,NaT,NaN,NaN,NaN,NaN,...,False,False,False,1,2023-03-15,396.0,NaN,NaN,NaN,NaN


In [200]:
## Chase's code

#create columns that show if a person uses a product
dataframe["uses_invest"] = ~dataframe['INV_OPEN_DATE'].isna() 
dataframe["uses_money"] = ~dataframe['MONEY_ACCOUNT_OPEN_DATE'].isna()
dataframe["uses_relay"] = ~dataframe['RELAY_ORIGINAL_DATE'].isna() 
dataframe["uses_loan"] = ~dataframe['ORIGINATION_DATE'].isna()
dataframe["uses_credit"] = ~dataframe['ONBOARDING_SUCCESS_DATE'].isna()

#create column that shows how many products a person uses
dataframe["num_products"] = (dataframe[['uses_invest', 'uses_money', 'uses_relay', 'uses_loan', 'uses_credit']] == True).sum(axis=1)

# create a column that shows how long a person has used a product. If they have never used a product, it will be NaN

dataframe['date_of_creating_new_columns'] = pd.to_datetime('today').date()

dataframe['INV_OPEN_DATE'] = pd.to_datetime(dataframe['INV_OPEN_DATE']).dt.date
dataframe['days_with_invest'] = (dataframe['date_of_creating_new_columns'] - dataframe['INV_OPEN_DATE']).dt.days

dataframe['MONEY_ACCOUNT_OPEN_DATE'] = pd.to_datetime(dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.date
dataframe['days_with_money'] = (dataframe['date_of_creating_new_columns'] - dataframe['MONEY_ACCOUNT_OPEN_DATE']).dt.days

dataframe['RELAY_ORIGINAL_DATE'] = pd.to_datetime(dataframe['RELAY_ORIGINAL_DATE']).dt.date
dataframe['days_with_relay'] = (dataframe['date_of_creating_new_columns'] - dataframe['RELAY_ORIGINAL_DATE']).dt.days

dataframe['ORIGINATION_DATE'] = pd.to_datetime(dataframe['ORIGINATION_DATE']).dt.date
dataframe['days_with_loan'] = (dataframe['date_of_creating_new_columns'] - dataframe['ORIGINATION_DATE']).dt.days

dataframe['ONBOARDING_SUCCESS_DATE'] = pd.to_datetime(dataframe['ONBOARDING_SUCCESS_DATE']).dt.date
dataframe['days_with_credit'] = (dataframe['date_of_creating_new_columns'] - dataframe['ONBOARDING_SUCCESS_DATE']).dt.days

# create a column 'first_product' that shows the first product a person used, and a column 'second_product' that shows the second product a person used. If they have never used a second product, it will be NaN

dataframe['first_product'] = np.nan
dataframe['second_product'] = np.nan

first_product_list = []
second_product_list = []

for i in tqdm(range(len(dataframe))):
    #create a list of the products that the user has
    products = []
    if dataframe.loc[i, 'uses_relay'] == True:
        products.append('relay')
    if dataframe.loc[i, 'uses_money'] == True:
        products.append('money')
    if dataframe.loc[i, 'uses_invest'] == True:
        products.append('invest')
    if dataframe.loc[i, 'uses_loan'] == True:
        products.append('loan')
    if dataframe.loc[i, 'uses_credit'] == True:
        products.append('credit')

    

    #sort the list of products by the number of days the user has used the product
    products.sort(key=lambda x: dataframe.loc[i, f'days_with_{x}'])

    #add the first two products to the list
    if len(products) >= 1:
        first_product_list.append(products[0])
    else:
        first_product_list.append(np.nan)
    if len(products) >= 2:
        second_product_list.append(products[1])
    else:
        second_product_list.append(np.nan)
        

dataframe['first_product'] = first_product_list
dataframe['second_product'] = second_product_list

100%|██████████| 3172773/3172773 [03:34<00:00, 14785.17it/s]


In [229]:
def analyze_relay(product):
    df = dataframe[dataframe['first_product'] == 'relay']
    cols = list(df.columns[13:21]) # relay columns
    cols.append('response') # response variable

    # identify the desired target product
    if(product == 'PL'):
        drop_columns = ['ACTIVE', 'PL_ACTIVE_IND']
        df['response'] = (dataframe['second_product'] == 'loan')

    if(product == 'Money'):
        drop_columns = ['ACTIVE', 'BANKING_ACTIVE_IND']
        df['response'] = (dataframe['second_product'] == 'money')

    df = df[cols]
    df = df.drop(columns=drop_columns)
    df.fillna(False,inplace=True)

    #downsample the data to balance the response variable (uses_money) (majority are users that do not use money and only use 1 product)
    #df = analyze_relay('PL')
    # Separate majority class (uses_money = 0 and num_products = 1)
    data_majority = df[df['response'] == False]
    data_minority = df[df['response'] == True]

    # Downsample majority class
    data_majority_downsampled = resample(data_majority, 
                                        replace=False,    # sample without replacement
                                        n_samples=len(data_minority),     # to match minority class
                                        random_state=123) # reproducible results

    # Combine minority class with downsampled majority class
    data_downsampled = pd.concat([data_majority_downsampled, data_minority])

    ## model
    X_train, X_test, y_train, y_test = train_test_split(data_downsampled[data_downsampled.columns[:-1]], data_downsampled['response'], test_size=0.33, random_state=34)
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # make predictions
    y_pred = model.predict(X_test)

    # check the accuracy
    accuracy = "ACCURACY: "+str(accuracy_score(y_test, y_pred))

    # find coefficients
    coefs = pd.DataFrame(zip(X_train.columns, np.transpose(model.coef_)), columns=['features', 'coef']) 

    # normalizing
    #for i in range(coefs.shape[0]):
        #coefs.iloc[i, 1] = math.exp(coefs.iloc[i, 1]) / (1 + math.exp(coefs.iloc[i, 1]))

    return coefs, accuracy, model, df



In [230]:
pl = analyze_relay('PL')
print(pl[1])
pl[0]

/var/folders/cw/hgq_4j310_x0341w_fpbn62m0000gp/T/ipykernel_35408/2615781308.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['response'] = (dataframe['second_product'] == 'loan')


ACCURACY: 0.5647418329978415


,features,coef
0,SL_ACTIVE_IND,[0.48641610557939813]
1,MORTGAGE_ACTIVE_IND,[1.1968633456346518]
2,BANKING_ACTIVE_IND,[-0.5239001288492082]
3,INVEST_ACTIVE_IND,[-0.4552700639267766]
4,CC_ACTIVE_IND,[0.25556875659236594]
5,CSM_ACTIVE_IND,[1.072133492141068]


potential issue with people being able to start relay tracking after they open the product of choice. so like they start tracking an additional product after they have already opened the app

In [233]:
# Display new class counts
data = pl[3]
data['response'].value_counts()

False    1519096
True      124947
Name: response, dtype: int64

In [226]:
money = analyze_relay('Money')
print(money[1])
money[0]

/var/folders/cw/hgq_4j310_x0341w_fpbn62m0000gp/T/ipykernel_35408/4148968243.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['response'] = (dataframe['second_product'] == 'money')


ACCURACY: 0.5449634088668371


,features,coef
0,SL_ACTIVE_IND,[-0.4805925804424388]
1,PL_ACTIVE_IND,[-0.7771194223773039]
2,MORTGAGE_ACTIVE_IND,[-0.0063535631592757675]
3,INVEST_ACTIVE_IND,[0.5220655035450114]
4,CC_ACTIVE_IND,[0.033386447801360235]
5,CSM_ACTIVE_IND,[0.2958501521334613]


In [232]:
money[3]['response'].value_counts()

False    1455018
True      189025
Name: response, dtype: int64